In [2]:
import requests

In [3]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open GIS Data

In [44]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [46]:
states_df["NAME"] = states_df["NAME"].str.lower()

## Get Population Data

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
}

In [7]:
r = requests.get("https://worldpopulationreview.com/states", headers=headers)

In [9]:
soup = BeautifulSoup(r.text, "html.parser")
tbodys = soup.find_all("tbody", {"class": "relative z-10 text-sm"})
population_table = tbodys[0]

In [40]:
population_list = []
for tr in population_table.find_all("tr"):
    tds = tr.find_all("td")
    population_dict = {
        "NAME": tr.find("a").text.lower(),
        "rank": tds[0].text,
        "pop2024": int(tds[1].text.replace(",", "")),
        "pop2023": int(tds[2].text.replace(",", "")),
        "change": float(tds[3].find("span").text.replace("%", "")),
    }
    population_list.append(population_dict)

for tr in tbodys[1].find_all("tr"):
    tds = tr.find_all("td")
    population_dict = {
        "NAME": tr.find("a").text.lower(),
        "pop2024": int(tds[1].text.replace(",", "")),
        "pop2023": int(tds[2].text.replace(",", "")),
        "change": float(tds[3].find("span").text.replace("%", "")),
    }
    population_list.append(population_dict)

In [41]:
population_df = pd.DataFrame(
    population_list, columns=["NAME", "pop2024", "pop2023", "change"]
)

## Merge Data

In [49]:
states_with_population_gdf = states_df.merge(population_df, on="NAME", how="right")[
    ["geometry", "pop2023", "pop2024", "change"]
]

In [51]:
states_with_population_gdf["difference"] = (
    states_with_population_gdf["pop2024"] - states_with_population_gdf["pop2023"]
)

In [52]:
states_with_population_gdf = states_with_population_gdf.to_crs(9311)

In [53]:
states_with_population_gdf.to_file("data/population.gpkg")